# <font color='blue'>Lab 11 - CKCS 113 Intro to Machine Learning</font>


**Following references are used for this module**
* Dataset: https://archive.ics.uci.edu/ml/datasets/adult
* https://docs.databricks.com/applications/machine-learning/mllib/binary-classification-mllib-pipelines.html

**Installing spark library and setting the Java envoirement**

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 52.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=72a931a29ec2182d0a8968de5107ea5667df57e97db6576c61d7284f725fe1fc
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


**Import the necessary libraries and setting the spark session**

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LogisticRegressionExample").getOrCreate()

**Upload the BostonHousing.csv file**

In [3]:
from google.colab import files
uploaded = files.upload()

Saving adult.data to adult.data


**Read the adult.data file into a dataset and name the columns**


In [0]:
dataset = spark.read.format("csv").option("header","false").load("adult.data").toDF('age',
  'workclass',  'fnlwgt',  'education',  'education_num',  'marital_status',  'occupation',
  'relationship',  'race',  'sex',  'capital_gain',  'capital_loss',  'hours_per_week',  'native_country',  'income')

**Print the schema of the dataset using describe and printschema functions**

In [5]:
dataset.printSchema()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: string (nullable = true)
 |-- capital_loss: string (nullable = true)
 |-- hours_per_week: string (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)



**Print first 5 records of the dataset**


In [6]:
dataset.head(5)

[Row(age='39', workclass=' State-gov', fnlwgt=' 77516', education=' Bachelors', education_num=' 13', marital_status=' Never-married', occupation=' Adm-clerical', relationship=' Not-in-family', race=' White', sex=' Male', capital_gain=' 2174', capital_loss=' 0', hours_per_week=' 40', native_country=' United-States', income=' <=50K'),
 Row(age='50', workclass=' Self-emp-not-inc', fnlwgt=' 83311', education=' Bachelors', education_num=' 13', marital_status=' Married-civ-spouse', occupation=' Exec-managerial', relationship=' Husband', race=' White', sex=' Male', capital_gain=' 0', capital_loss=' 0', hours_per_week=' 13', native_country=' United-States', income=' <=50K'),
 Row(age='38', workclass=' Private', fnlwgt=' 215646', education=' HS-grad', education_num=' 9', marital_status=' Divorced', occupation=' Handlers-cleaners', relationship=' Not-in-family', race=' White', sex=' Male', capital_gain=' 0', capital_loss=' 0', hours_per_week=' 40', native_country=' United-States', income=' <=50K

In [7]:
dataset.describe()

DataFrame[summary: string, age: string, workclass: string, fnlwgt: string, education: string, education_num: string, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: string, capital_loss: string, hours_per_week: string, native_country: string, income: string]

## Changing Data Types

In [8]:
dataset = dataset.withColumn("age", dataset["age"].cast("Integer"))
dataset = dataset.withColumn("fnlwgt", dataset["fnlwgt"].cast("Integer"))
dataset = dataset.withColumn("education_num", dataset["education_num"].cast("Integer"))
dataset = dataset.withColumn("capital_gain", dataset["capital_gain"].cast("Integer"))
dataset = dataset.withColumn("capital_loss", dataset["capital_loss"].cast("Integer"))
dataset = dataset.withColumn("hours_per_week", dataset["hours_per_week"].cast("Integer"))
dataset.printSchema()


root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)



**Make the predictions by considering the numeric column only**
